In [88]:
import numpy as np
import importlib
import Layout
import Greedy

import copy 

#Reset libreria
importlib.reload(px)
importlib.reload(Greedy)

def showtime2(path):
    layout=px.read(path)
    #print ("Estado inicial: ")
    #for s in layout.stacks:
    #    print(s)
    #print()
    seleccionada = SSRC(layout)
 
    while seleccionada != -1:
        #print("SELECCIONADA:", seleccionada)
        
        #Se calcula cuantos contenedores deben retirarse
        columna=copy.deepcopy(layout.stacks[seleccionada])
        nb_unsorted_elements = px.remove_until(columna)

        #Se calculan los ranks
        rank = Greedy.get_ranks(layout.stacks[seleccionada])
        #print(rank)

        #Se retiran los elementos de la columna
        SDpp(layout, seleccionada, rank, nb_unsorted_elements)
        #for s in layout.stacks:
        #    print(s)
        #print("\n")

        #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
        for item in layout.stacks[seleccionada]:
            rank.pop(item, None)


        SFpp(layout,seleccionada,rank,nb_unsorted_elements)
        #for s in layout.stacks:
        #    print(s)
        #print("\n")

        #Se vuelve a seleccionar una columna
        seleccionada = SSRC(layout)

    #print("SELECCIONADA:", seleccionada)
    print(len(layout.moves),"\t",layout.unsorted_stacks)
    return len(layout.moves)
    #print()

In [58]:
#SELECT SEMI RANDOM COLUMN
def SSRC(layout):
    
    size=len(layout.stacks)
    best=-1000000000
    select=-1
    
    posibles = []

    for i in range (0,size):

        aux = layout.is_sorted_stack(i)

        if aux == False:
            posibles.append(i)

    #Me di cuenta que con el que mayor problemas tiene tenemos menos problemas 
    #(necesita menos espacio al contrario del que tenga menos problemas)

    #Si no hay posibles se retorna -1 
    if len(posibles) == 0:
        return -1    
    
    #En casoq ue si hay posibles
    
    for i in posibles:
        aux = px.comparacion(layout.stacks[i])
        if aux > best:
            best = aux
            select = i 
    return select   

In [69]:
def SDpp(layout, s_o, rank, n_elements=100):
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    print(ss_o)
    
    fixed_elements = np.maximum(len(ss_o)-n_elements,0)
    
    while len(layout.stacks[s_o]) > fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            Greedy.fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)

In [92]:
import sys
def SFpp(layout, s_d, rank, n):
    ub = 100
    cont = 0
    for r in rank:
        while True:
            (s, pos) = Greedy.search_highest(layout, r, ub, s_d)  
            #print(r,ub)
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            #print(layout.stacks, s, pos, s_d)
            
            c=Greedy.force_move(layout, s, pos, s_d)
            if c == None: 
                ub=layout.stacks[s][pos]-1
                if ub<=r: 
                    print(ub+ " del stack " +s+ " no se puede volver a colocar en "+s_d);
                    sys.exit()
            else: 
                ub=r
                cont+=1
                if cont == n: return
                if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

In [95]:
import Greedy as gr
import importlib
importlib.reload(Greedy)

path="Instancias\\10-10\\data10-10-"

nmoves=0
for i in range(1,30):
    nmoves += showtime2(path+ str(i)+".dat")
    
print (nmoves/ 30.0);

[14, 37, 3, 90, 46, 31, 79, 43, 32, 68]
[60, 7, 53, 26, 76, 75, 51, 94, 92, 86, 3, 14]
[65, 63, 85, 55, 57, 27, 19, 4, 18, 29, 32]
[1, 24, 77, 35, 69, 42, 66, 17, 16, 7, 18, 19]
[78, 25, 48, 81, 59, 23, 47, 89, 4, 7, 16, 17]
[61, 41, 58, 50, 67, 80, 39, 28, 45, 7]
[15, 87, 62, 97, 93, 5, 98, 34, 36]
[30, 12, 11, 8, 9, 22, 10, 5]
[2, 84, 96, 56, 71, 49]
187 	 0
[5, 76, 67, 84, 45, 35, 50, 99, 12, 21]
[66, 10, 20, 59, 60, 95, 97, 62, 27, 3, 12]
[39, 14, 36, 82, 19, 7, 53, 74, 42, 2, 5]
[70, 57, 96, 71, 75, 68, 55, 69, 79, 2, 7, 14]
[43, 44, 25, 31, 24, 28, 6, 37, 85, 7, 2]
[93, 47, 89, 18, 17, 87, 40, 100, 1, 10]
[26, 83, 41, 81, 8, 2, 7, 6, 1]
[13, 56, 32, 30, 73, 51, 61, 90]
[98, 80, 46, 23, 4, 1, 13, 32]
[98, 80, 46, 58, 32, 23]
[98, 80, 92, 58, 46, 32]
[90, 88, 73, 61, 56, 52, 51, 30, 13, 1, 4, 23]
200 	 0
[55, 8, 34, 48, 57, 1, 53, 11, 73, 38]
[83, 61, 90, 58, 49, 92, 5, 19, 44, 59, 11, 1]
[15, 94, 69, 72, 79, 4, 17, 40, 10, 20, 34, 8]
[30, 86, 46, 82, 80, 51, 23, 47, 81, 8, 10, 4]
